In [38]:
import numpy as np
import pandas as pd
import joblib



In [39]:
# Reading aggregated dataset
df = pd.read_csv('/home/lavrenchuk/TS_Project_Favorita/data/processed/data.csv')
display(df.tail())

,date,unit_sales
1682,2017-08-11,80783.227
1683,2017-08-12,77928.860
1684,2017-08-13,92090.859
1685,2017-08-14,69852.929
1686,2017-08-15,80080.046


In [40]:
hol = pd.read_csv("/home/lavrenchuk/TS_Project_Favorita/data/raw/holidays_events.csv")
hol["date"] = pd.to_datetime(hol["date"])
hol = hol[hol["type"].isin(["Holiday", "Additional", "Bridge"])] #Holiday + Additional + Bridge => is_holiday
display(hol.tail())

,date,type,locale,locale_name,description,transferred
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [41]:
hol_simple = hol[["date"]].drop_duplicates()
hol_simple["is_holiday"] = 1
display(hol_simple.tail())
hol_simple = hol_simple.set_index("date")

,date,is_holiday
344,2017-12-22,1
346,2017-12-23,1
347,2017-12-24,1
348,2017-12-25,1
349,2017-12-26,1


In [42]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index("date")
#Таргет:
df['unit_sales']    #y
df['unit_sales_log']= np.log1p(df['unit_sales'])   #y_log

#Календарні фічі:
df['month'] = df.index.month 
df['day_of_month'] = df.index.day
df['weekday'] = df.index.dayofweek  # 0=Mon, 6=Sun
df["is_workday"]   = (df["weekday"] <= 4).astype(int)  # [Mon, Tue, Wed, Thu, Fri] => 1, [Sat, Sun]-> 0.
# df["is_holiday"]

#Лагові фічі:
df = df.sort_index()
df['lag_log_1'] = df['unit_sales_log'].shift(1)
df['lag_log_7'] = df['unit_sales_log'].shift(7)
df['lag_log_14'] = df['unit_sales_log'].shift(14)
df['lag_log_28'] = df['unit_sales_log'].shift(28)

#Рухоме середнє
df['roll_mean_7']  = df['unit_sales_log'].rolling(window=7).mean().shift(1)   #shift як запобігання leakage data
df['roll_mean_14'] = df['unit_sales_log'].rolling(window=14).mean().shift(1)
df['roll_mean_28'] = df['unit_sales_log'].rolling(window=28).mean().shift(1)
df['roll_mean_56'] = df['unit_sales_log'].rolling(window=56).mean().shift(1)

df['roll_std_7']   = df['unit_sales_log'].rolling(window=7).std().shift(1)
df['roll_std_28']  = df['unit_sales_log'].rolling(window=28).std().shift(1)

#Циклічні змінні
df["dow_sin"] = np.sin(2*np.pi*df['weekday']/7)
df["dow_cos"] = np.cos(2*np.pi*df['weekday']/7)
df["year_sin"] = np.sin(2*np.pi*df["month"]/12)
df["year_cos"] = np.cos(2*np.pi*df["month"]/12)

print(df.index)   #control of index
display(df.tail())

DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-05',
               '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09',
               '2013-01-10', '2013-01-11',
               ...
               '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09',
               '2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', length=1687, freq=None)


,unit_sales,unit_sales_log,month,day_of_month,weekday,is_workday,lag_log_1,lag_log_7,lag_log_14,lag_log_28,roll_mean_7,roll_mean_14,roll_mean_28,roll_mean_56,roll_std_7,roll_std_28,dow_sin,dow_cos,year_sin,year_cos
date,,,,,,,,,,,,,,,,,,,,
2017-08-11,80783.227,11.299537,8,11,4,1,10.984767,11.172635,11.192696,11.086023,11.213203,11.295054,11.241135,11.230961,0.204764,0.185977,-0.433884,-0.900969,-0.866025,-0.5
2017-08-12,77928.860,11.263564,8,12,5,0,11.299537,11.399004,11.469589,11.365008,11.231332,11.302685,11.248761,11.233187,0.206186,0.183745,-0.974928,-0.222521,-0.866025,-0.5
2017-08-13,92090.859,11.430542,8,13,6,0,11.263564,11.571467,11.615288,11.574843,11.211983,11.287969,11.245138,11.229576,0.193813,0.182363,-0.781831,0.623490,-0.866025,-0.5
2017-08-14,69852.929,11.154162,8,14,0,1,11.430542,11.195912,11.342291,11.196703,11.191851,11.274773,11.239984,11.230797,0.153342,0.174573,0.000000,1.000000,-0.866025,-0.5
2017-08-15,80080.046,11.290794,8,15,1,1,11.154162,11.110238,11.509225,11.098969,11.185887,11.261335,11.238465,11.229696,0.153969,0.175148,0.781831,0.623490,-0.866025,-0.5


In [43]:
df = df.merge(hol_simple, left_index=True, right_index=True, how="left")
df["is_holiday"] = df["is_holiday"].fillna(0).astype(int)
df["before_holiday"] = df["is_holiday"].shift(-1).fillna(0).astype(int)
df["after_holiday"]  = df["is_holiday"].shift(1).fillna(0).astype(int)
df = df.sort_index()
display(df.tail())

,unit_sales,unit_sales_log,month,day_of_month,weekday,is_workday,lag_log_1,lag_log_7,lag_log_14,lag_log_28,...,roll_mean_56,roll_std_7,roll_std_28,dow_sin,dow_cos,year_sin,year_cos,is_holiday,before_holiday,after_holiday
date,,,,,,,,,,,,,,,,,,,,,
2017-08-11,80783.227,11.299537,8,11,4,1,10.984767,11.172635,11.192696,11.086023,...,11.230961,0.204764,0.185977,-0.433884,-0.900969,-0.866025,-0.5,0,0,1
2017-08-12,77928.860,11.263564,8,12,5,0,11.299537,11.399004,11.469589,11.365008,...,11.233187,0.206186,0.183745,-0.974928,-0.222521,-0.866025,-0.5,0,0,0
2017-08-13,92090.859,11.430542,8,13,6,0,11.263564,11.571467,11.615288,11.574843,...,11.229576,0.193813,0.182363,-0.781831,0.623490,-0.866025,-0.5,0,0,0
2017-08-14,69852.929,11.154162,8,14,0,1,11.430542,11.195912,11.342291,11.196703,...,11.230797,0.153342,0.174573,0.000000,1.000000,-0.866025,-0.5,0,1,0
2017-08-15,80080.046,11.290794,8,15,1,1,11.154162,11.110238,11.509225,11.098969,...,11.229696,0.153969,0.175148,0.781831,0.623490,-0.866025,-0.5,1,0,0


In [44]:
df_features = df.copy()
joblib.dump(
    df_features,
    "/home/lavrenchuk/TS_Project_Favorita/data/processed/features_agg.joblib"
)

['/home/lavrenchuk/TS_Project_Favorita/data/processed/features_agg.joblib']